In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
train = pd.read_csv('train_en.txt', sep='\t')
test = pd.read_csv('test_en.txt', sep='\t')

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
sentences = train['Sentence'].values.tolist()
labels = train['Style'].values.tolist()

train_encodings = tokenizer(
    sentences,
    max_length=15,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_attention_mask=True
)

train_input_ids = train_encodings['input_ids']
train_attention_masks = train_encodings['attention_mask']

In [5]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [6]:
encoded_labels = torch.tensor(encoded_labels)

In [7]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, encoded_labels)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [8]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
optimizer = AdamW(model.parameters(), lr=0.0001, weight_decay=0.0005)

In [11]:
from tqdm import tqdm

epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", dynamic_ncols=True)

    for batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)


        optimizer.zero_grad()


        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits


        loss.backward()
        optimizer.step()


        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_predictions += labels.size(0)


        progress_bar.set_postfix(loss=loss.item(), accuracy=correct_predictions/total_predictions)

        running_loss += loss.item()


    avg_loss = running_loss / len(train_dataloader)
    accuracy = correct_predictions / total_predictions


    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.4f}")


Epoch 1/50: 100%|██████████| 196/196 [00:33<00:00,  5.88it/s, accuracy=tensor(0.9015, device='cuda:0'), loss=0.112]


Epoch 1/50 - Loss: 0.2290 - Accuracy: 0.9015


Epoch 2/50: 100%|██████████| 196/196 [00:32<00:00,  6.04it/s, accuracy=tensor(0.9306, device='cuda:0'), loss=0.181]


Epoch 2/50 - Loss: 0.1495 - Accuracy: 0.9306


Epoch 3/50: 100%|██████████| 196/196 [00:33<00:00,  5.90it/s, accuracy=tensor(0.9406, device='cuda:0'), loss=0.0797]


...

Epoch 48/50 - Loss: 0.0463 - Accuracy: 0.9706


Epoch 49/50: 100%|██████████| 196/196 [00:34<00:00,  5.72it/s, accuracy=tensor(0.9693, device='cuda:0'), loss=0.0836]


Epoch 49/50 - Loss: 0.0510 - Accuracy: 0.9693


Epoch 50/50: 100%|██████████| 196/196 [00:34<00:00,  5.73it/s, accuracy=tensor(0.9690, device='cuda:0'), loss=0.132]

Epoch 50/50 - Loss: 0.0517 - Accuracy: 0.9690


In [12]:
model.eval()
test_sentences = test['Sentence'].values.tolist()
test_encodings = tokenizer(
    test_sentences,
    max_length=15,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_attention_mask=True
)

test_input_ids = test_encodings['input_ids']
test_attention_masks = test_encodings['attention_mask']

In [13]:
with torch.no_grad():
    outputs = model(input_ids=test_input_ids.to(device), attention_mask=test_attention_masks.to(device))
    logits = outputs.logits

predictions = torch.argmax(logits, dim=1).cpu().numpy()

true_labels = label_encoder.transform(test['Style'].values.tolist())


d_accuracy = accuracy_score(true_labels, predictions)
d_precision = precision_score(true_labels, predictions, average='weighted')
d_recall = recall_score(true_labels, predictions, average='weighted')
d_f1 = f1_score(true_labels, predictions, average='weighted')


print(f"Test Accuracy: {d_accuracy:.4f}")
print(f"Test Precision: {d_precision:.4f}")
print(f"Test Recall: {d_recall:.4f}")
print(f"Test F1 Score: {d_f1:.4f}")

Test Accuracy: 0.9112
Test Precision: 0.9139
Test Recall: 0.9112
Test F1 Score: 0.9111


In [14]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [15]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
sentences = train['Sentence'].values.tolist()
labels = train['Style'].values.tolist()

train_encodings = tokenizer(
    sentences,
    max_length=15,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_attention_mask=True
)

In [17]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = torch.tensor(encoded_labels)

In [18]:
train_input_ids = train_encodings['input_ids']
train_attention_masks = train_encodings['attention_mask']

In [19]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, encoded_labels)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [21]:
optimizer = AdamW(model.parameters(), lr=0.0001, weight_decay=0.0005)

In [22]:
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", dynamic_ncols=True)

    for batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()


        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_predictions += labels.size(0)


        progress_bar.set_postfix(loss=loss.item(), accuracy=correct_predictions/total_predictions)

        running_loss += loss.item()


    avg_loss = running_loss / len(train_dataloader)
    accuracy = correct_predictions / total_predictions


    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.4f}")



Epoch 1/50: 100%|██████████| 196/196 [01:08<00:00,  2.85it/s, accuracy=tensor(0.9077, device='cuda:0'), loss=0.249]


Epoch 1/50 - Loss: 0.1919 - Accuracy: 0.9077


Epoch 2/50: 100%|██████████| 196/196 [01:08<00:00,  2.86it/s, accuracy=tensor(0.9351, device='cuda:0'), loss=0.271]


Epoch 2/50 - Loss: 0.1288 - Accuracy: 0.9351


Epoch 3/50: 100%|██████████| 196/196 [01:08<00:00,  2.85it/s, accuracy=tensor(0.9413, device='cuda:0'), loss=0.17]


...

Epoch 48/50 - Loss: 0.6936 - Accuracy: 0.5017


Epoch 49/50: 100%|██████████| 196/196 [01:08<00:00,  2.86it/s, accuracy=tensor(0.5016, device='cuda:0'), loss=0.698]


Epoch 49/50 - Loss: 0.6935 - Accuracy: 0.5016


Epoch 50/50: 100%|██████████| 196/196 [01:08<00:00,  2.86it/s, accuracy=tensor(0.5023, device='cuda:0'), loss=0.694]

Epoch 50/50 - Loss: 0.6935 - Accuracy: 0.5023


In [23]:
model.eval()
test_sentences = test['Sentence'].values.tolist()
test_encodings = tokenizer(
    test_sentences,
    max_length=15,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_attention_mask=True
)

test_input_ids = test_encodings['input_ids']
test_attention_masks = test_encodings['attention_mask']

with torch.no_grad():
    outputs = model(input_ids=test_input_ids.to(device), attention_mask=test_attention_masks.to(device))
    logits = outputs.logits

r_accuracy = accuracy_score(true_labels, predictions)
r_precision = precision_score(true_labels, predictions, average='weighted')
r_recall = recall_score(true_labels, predictions, average='weighted')
r_f1 = f1_score(true_labels, predictions, average='weighted')


print(f"Test Accuracy: {r_accuracy:.4f}")
print(f"Test Precision: {r_precision:.4f}")
print(f"Test Recall: {r_recall:.4f}")
print(f"Test F1 Score: {r_f1:.4f}")

Test Accuracy: 0.9112
Test Precision: 0.9139
Test Recall: 0.9112
Test F1 Score: 0.9111


In [24]:
print(f"DistilBERT - Test Accuracy: {d_accuracy:.4f}")
print(f"DistilBERT - Test Precision: {d_precision:.4f}")
print(f"DistilBERT - Test Recall: {d_recall:.4f}")
print(f"DistilBERT - Test F1 Score: {d_f1:.4f}")


print(f"RoBERTa - Test Accuracy: {r_accuracy:.4f}")
print(f"RoBERTa - Test Precision: {r_precision:.4f}")
print(f"RoBERTa - Test Recall: {r_recall:.4f}")
print(f"RoBERTa - Test F1 Score: {r_f1:.4f}")


print("\nComparison of Models:")


if d_accuracy > r_accuracy:
    print(f"DistilBERT has better accuracy ({d_accuracy:.4f}) compared to RoBERTa ({r_accuracy:.4f}).")
else:
    print(f"RoBERTa has better accuracy ({r_accuracy:.4f}) compared to DistilBERT ({d_accuracy:.4f}).")


if d_precision > r_precision:
    print(f"DistilBERT has better precision ({d_precision:.4f}) compared to RoBERTa ({r_precision:.4f}).")
else:
    print(f"RoBERTa has better precision ({r_precision:.4f}) compared to DistilBERT ({d_precision:.4f}).")

if d_recall > r_recall:
    print(f"DistilBERT has better recall ({d_recall:.4f}) compared to RoBERTa ({r_recall:.4f}).")
else:
    print(f"RoBERTa has better recall ({r_recall:.4f}) compared to DistilBERT ({d_recall:.4f}).")


if d_f1 > r_f1:
    print(f"DistilBERT has better F1 score ({d_f1:.4f}) compared to RoBERTa ({r_f1:.4f}).")
else:
    print(f"RoBERTa has better F1 score ({r_f1:.4f}) compared to DistilBERT ({d_f1:.4f}).")

DistilBERT - Test Accuracy: 0.9112
DistilBERT - Test Precision: 0.9139
DistilBERT - Test Recall: 0.9112
DistilBERT - Test F1 Score: 0.9111
RoBERTa - Test Accuracy: 0.9112
RoBERTa - Test Precision: 0.9139
RoBERTa - Test Recall: 0.9112
RoBERTa - Test F1 Score: 0.9111

Comparison of Models:
RoBERTa has better accuracy (0.9112) compared to DistilBERT (0.9112).
RoBERTa has better precision (0.9139) compared to DistilBERT (0.9139).
RoBERTa has better recall (0.9112) compared to DistilBERT (0.9112).
RoBERTa has better F1 score (0.9111) compared to DistilBERT (0.9111).


### Заклучок: моделите се  подобри отколку тие во првата задача.